.. _nb_subset_selection:

## Subset Selection Problem

A genetic algorithm can be used to approach subset selection problems by defining custom operators. In general a metaheuristic algorithm might not be the ultimate goal to implement in a real-world scenario, however, it might be useful to investigate patterns or characteristics of possible good subsets. 
Let us consider a simple toy problem where we have to select numbers from a list. For every solution exactly 10 numbers have to be selected that their sum is minimized.
For subset selection problem a binary encoding can be used where 1 indicates a number is picked. In our problem formulation the list of numbers is represented by $L$ and the binary encoded variable by $x$.


\begin{align}
\begin{split}
\min f(x) & = & \sum_{k=1}^{n} L_k \cdot x_k\\[2mm]
\text{s.t.} \quad g(x)  & = & (\sum_{k=1}^{n} x_k - 10)^2\\[2mm]
\end{split}
\end{align}


As shown above, the equality constraint is handled by making sure $g(x)$ can only be zero if exactly 10 numbers are chosen.
The problem can be implemented as follows:

In [1]:
import numpy as np
from pymoo.model.problem import Problem

class SubsetProblem(Problem):
    def __init__(self,
                 L,
                 n_max
                 ):
        super().__init__(n_var=len(L), n_obj=1, n_constr=1, elementwise_evaluation=True)
        self.L = L
        self.n_max = n_max

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum(self.L[x])
        out["G"] = (self.n_max - np.sum(x)) ** 2
    
    
# create the actual problem to be solved
np.random.seed(1)
L = np.array([np.random.randint(100) for _ in range(100)])
n_max = 10
problem = SubsetProblem(L, n_max)


The customization requires to write custom operators in order to solve this problem efficiently. We recommend to consider the feasibility directly in the evolutionary operators, because otherwise most of the time infeasible solutions will be processed.
The sampling creates randomly solution where the subset constraint will always be satisfied. 
The mutation randomly removes a number and chooses another one. The crossover, first takes the common numbers of both parents and then randomly picks either from the first or from the second parent until enough numbers are picked.

In [2]:
from pymoo.model.crossover import Crossover
from pymoo.model.mutation import Mutation
from pymoo.model.sampling import Sampling


class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
        X = np.full((n_samples, problem.n_var), False, dtype=np.bool)

        for k in range(n_samples):
            I = np.random.permutation(problem.n_var)[:problem.n_max]
            X[k, I] = True

        return X


class BinaryCrossover(Crossover):
    def __init__(self):
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        n_parents, n_matings, n_var = X.shape

        _X = np.full((self.n_offsprings, n_matings, problem.n_var), False)

        for k in range(n_matings):
            p1, p2 = X[0, k], X[1, k]

            both_are_true = np.logical_and(p1, p2)
            _X[0, k, both_are_true] = True

            n_remaining = problem.n_max - np.sum(both_are_true)

            I = np.where(np.logical_xor(p1, p2))[0]

            S = I[np.random.permutation(len(I))][:n_remaining]
            _X[0, k, S] = True

        return _X


class MyMutation(Mutation):
    def _do(self, problem, X, **kwargs):
        for i in range(X.shape[0]):
            X[i, :] = X[i, :]
            is_false = np.where(np.logical_not(X[i, :]))[0]
            is_true = np.where(X[i, :])[0]
            X[i, np.random.choice(is_false)] = True
            X[i, np.random.choice(is_true)] = False

        return X

After having defined the operators a genetic algorithm can be initialized.

In [3]:
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.optimize import minimize

algorithm = GA(
    pop_size=100,
    sampling=MySampling(),
    crossover=BinaryCrossover(),
    mutation=MyMutation(),
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               ('n_gen', 60),
               seed=1,
               verbose=True)

print("Function value: %s" % res.F[0])
print("Subset:", np.where(res.X)[0])


n_gen |  n_eval |   cv (min)   |   cv (avg)   |     fopt     |     favg    
    1 |     100 |  0.00000E+00 |  0.00000E+00 |          258 |  4.43940E+02
    2 |     200 |  0.00000E+00 |  0.00000E+00 |          205 |  3.54340E+02
    3 |     300 |  0.00000E+00 |  0.00000E+00 |          175 |  3.00390E+02
    4 |     400 |  0.00000E+00 |  0.00000E+00 |          161 |  2.52240E+02
    5 |     500 |  0.00000E+00 |  0.00000E+00 |          124 |  2.18240E+02
    6 |     600 |  0.00000E+00 |  0.00000E+00 |          118 |  1.91860E+02
    7 |     700 |  0.00000E+00 |  0.00000E+00 |          118 |  1.70050E+02
    8 |     800 |  0.00000E+00 |  0.00000E+00 |           85 |  1.49630E+02
    9 |     900 |  0.00000E+00 |  0.00000E+00 |           85 |  1.39550E+02
   10 |    1000 |  0.00000E+00 |  0.00000E+00 |           85 |  1.30580E+02
   11 |    1100 |  0.00000E+00 |  0.00000E+00 |           74 |  1.19930E+02


   12 |    1200 |  0.00000E+00 |  0.00000E+00 |           74 |  1.11800E+02
   13 |    1300 |  0.00000E+00 |  0.00000E+00 |           74 |  1.06050E+02
   14 |    1400 |  0.00000E+00 |  0.00000E+00 |           68 |  1.00100E+02
   15 |    1500 |  0.00000E+00 |  0.00000E+00 |           68 |  9.46300E+01
   16 |    1600 |  0.00000E+00 |  0.00000E+00 |           52 |  8.97600E+01
   17 |    1700 |  0.00000E+00 |  0.00000E+00 |           50 |  8.45100E+01
   18 |    1800 |  0.00000E+00 |  0.00000E+00 |           50 |  8.09000E+01
   19 |    1900 |  0.00000E+00 |  0.00000E+00 |           50 |  7.66900E+01
   20 |    2000 |  0.00000E+00 |  0.00000E+00 |           50 |  7.33700E+01
   21 |    2100 |  0.00000E+00 |  0.00000E+00 |           45 |  7.00100E+01
   22 |    2200 |  0.00000E+00 |  0.00000E+00 |           45 |  6.78900E+01
   23 |    2300 |  0.00000E+00 |  0.00000E+00 |           45 |  6.60300E+01


   24 |    2400 |  0.00000E+00 |  0.00000E+00 |           45 |  6.43500E+01
   25 |    2500 |  0.00000E+00 |  0.00000E+00 |           45 |  6.36500E+01
   26 |    2600 |  0.00000E+00 |  0.00000E+00 |           45 |  6.16100E+01
   27 |    2700 |  0.00000E+00 |  0.00000E+00 |           45 |  6.03300E+01
   28 |    2800 |  0.00000E+00 |  0.00000E+00 |           43 |  5.94900E+01
   29 |    2900 |  0.00000E+00 |  0.00000E+00 |           43 |  5.76600E+01
   30 |    3000 |  0.00000E+00 |  0.00000E+00 |           43 |  5.65100E+01
   31 |    3100 |  0.00000E+00 |  0.00000E+00 |           42 |  5.54700E+01
   32 |    3200 |  0.00000E+00 |  0.00000E+00 |           42 |  5.42800E+01
   33 |    3300 |  0.00000E+00 |  0.00000E+00 |           42 |  5.36700E+01
   34 |    3400 |  0.00000E+00 |  0.00000E+00 |           42 |  5.27000E+01
   35 |    3500 |  0.00000E+00 |  0.00000E+00 |           42 |  5.21800E+01


   36 |    3600 |  0.00000E+00 |  0.00000E+00 |           42 |  5.19700E+01
   37 |    3700 |  0.00000E+00 |  0.00000E+00 |           39 |  5.14400E+01
   38 |    3800 |  0.00000E+00 |  0.00000E+00 |           39 |  5.10500E+01
   39 |    3900 |  0.00000E+00 |  0.00000E+00 |           39 |  5.08500E+01
   40 |    4000 |  0.00000E+00 |  0.00000E+00 |           39 |  5.05500E+01
   41 |    4100 |  0.00000E+00 |  0.00000E+00 |           39 |  5.01200E+01
   42 |    4200 |  0.00000E+00 |  0.00000E+00 |           39 |  4.93800E+01
   43 |    4300 |  0.00000E+00 |  0.00000E+00 |           39 |  4.88000E+01
   44 |    4400 |  0.00000E+00 |  0.00000E+00 |           39 |  4.85700E+01
   45 |    4500 |  0.00000E+00 |  0.00000E+00 |           39 |  4.81400E+01
   46 |    4600 |  0.00000E+00 |  0.00000E+00 |           38 |  4.76700E+01
   47 |    4700 |  0.00000E+00 |  0.00000E+00 |           38 |  4.71800E+01


   48 |    4800 |  0.00000E+00 |  0.00000E+00 |           37 |  4.66100E+01
   49 |    4900 |  0.00000E+00 |  0.00000E+00 |           37 |  4.62700E+01
   50 |    5000 |  0.00000E+00 |  0.00000E+00 |           37 |  4.62000E+01
   51 |    5100 |  0.00000E+00 |  0.00000E+00 |           37 |  4.57600E+01
   52 |    5200 |  0.00000E+00 |  0.00000E+00 |           37 |  4.56000E+01
   53 |    5300 |  0.00000E+00 |  0.00000E+00 |           37 |  4.53400E+01
   54 |    5400 |  0.00000E+00 |  0.00000E+00 |           36 |  4.49200E+01
   55 |    5500 |  0.00000E+00 |  0.00000E+00 |           36 |  4.46700E+01
   56 |    5600 |  0.00000E+00 |  0.00000E+00 |           36 |  4.45100E+01
   57 |    5700 |  0.00000E+00 |  0.00000E+00 |           36 |  4.43800E+01
   58 |    5800 |  0.00000E+00 |  0.00000E+00 |           36 |  4.42300E+01
   59 |    5900 |  0.00000E+00 |  0.00000E+00 |           36 |  4.41400E+01


   60 |    6000 |  0.00000E+00 |  0.00000E+00 |           36 |  4.38100E+01
Function value: 36
Subset: [ 5  9 12 31 36 37 47 52 68 99]


Finally, we can compare the found subset with the optimum known simply through sorting:

In [4]:
opt = np.sort(np.argsort(L)[:n_max])
print("Optimal Subset:", opt)
print("Optimal Function Value: %s" % L[opt].sum())

Optimal Subset: [ 5  9 12 31 36 37 47 52 68 99]
Optimal Function Value: 36
